In [43]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [44]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device.type

'cpu'

In [45]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [46]:
# MNIST dataset (images and labels)
train_dataset = datasets.MNIST(
    root="/home/gopalakrishnanm/Projects/Pytorch_tutorial/data/mnist/",
    train=True,
    download=False,
    transform=ToTensor())
test_dataset = datasets.MNIST(
    root="/home/gopalakrishnanm/Projects/Pytorch_tutorial/data/mnist/",
    train=False,
    download=False,
    transform=ToTensor())

In [55]:
# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


# Let's check out what we've created
print(f"Dataloaders: {train_loader, test_loader}")
print(f"Length of train dataloader: {len(train_loader)} batches of {batch_size}")
print(f"Length of test dataloader: {len(test_loader)} batches of {batch_size}")

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7fb827f71690>, <torch.utils.data.dataloader.DataLoader object at 0x7fb821b989d0>)
Length of train dataloader: 600 batches of 100
Length of test dataloader: 100 batches of 100


In [57]:
# Check out what's inside the training dataloader
train_features_batch, train_labels_batch = next(iter(train_loader))
train_features_batch.shape, train_labels_batch.shape

(torch.Size([100, 1, 28, 28]), torch.Size([100]))

In [7]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes = 10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [11]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and Optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%100 == 0:
            print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}".format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.0919
Epoch [1/5], Step [200/600], Loss: 0.1268
Epoch [1/5], Step [300/600], Loss: 0.2364
Epoch [1/5], Step [400/600], Loss: 0.1204
Epoch [1/5], Step [500/600], Loss: 0.0785
Epoch [1/5], Step [600/600], Loss: 0.1410
Epoch [2/5], Step [100/600], Loss: 0.0689
Epoch [2/5], Step [200/600], Loss: 0.1891
Epoch [2/5], Step [300/600], Loss: 0.1498
Epoch [2/5], Step [400/600], Loss: 0.1187
Epoch [2/5], Step [500/600], Loss: 0.1536
Epoch [2/5], Step [600/600], Loss: 0.1093
Epoch [3/5], Step [100/600], Loss: 0.0811
Epoch [3/5], Step [200/600], Loss: 0.1359
Epoch [3/5], Step [300/600], Loss: 0.1674
Epoch [3/5], Step [400/600], Loss: 0.1464
Epoch [3/5], Step [500/600], Loss: 0.1899
Epoch [3/5], Step [600/600], Loss: 0.1340
Epoch [4/5], Step [100/600], Loss: 0.0842
Epoch [4/5], Step [200/600], Loss: 0.0805
Epoch [4/5], Step [300/600], Loss: 0.0688
Epoch [4/5], Step [400/600], Loss: 0.1096
Epoch [4/5], Step [500/600], Loss: 0.1373
Epoch [4/5], Step [600/600], Loss:

In [17]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        max_value, predicted = torch.max(outputs.data, 1) #computes the maximum value and its index along dimension 1
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
    print(len(test_dataset))

Test Accuracy of the model on the 10000 test images: 90.79 %
10000


In [19]:
print(labels.size(-1))

100
